In [1]:
# Import packages and set OPENAI Key access
import nltk
import json
#import langchain as lc
from getpass import getpass
import os
os.chdir('..')
from langchain.chat_models import ChatOpenAI
#from langchain.chains import create_extraction_chain
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
PROMED_DIR = os.path.join('..','WHO_DON','Data','ProMED')
WHO_DIR = os.path.join('..','WHO_DON','Data','WHO DON')
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
OPENAI_ORGANIZATION = os.environ['OPENAI_ORGANIZATION']
chatGPT4 = ChatOpenAI(openai_api_key=OPENAI_API_KEY, openai_organization= OPENAI_ORGANIZATION, temperature=0, model="gpt-4")
chatGPT35 = ChatOpenAI(openai_api_key=OPENAI_API_KEY, openai_organization= OPENAI_ORGANIZATION, temperature=0, model="gpt-3.5-turbo")


In [3]:
# Load 1st set of Promed data
D1 = json.load(open('../WHO_DON/Data/ProMED/promed_1.json'))

In [ ]:
# Headers that demonstrate that the extrations can't be made by parsing as the disease outbreak location is not 
# always at the same position. Need to use LLMs
import numpy as np
print(D1[0]['header'][0])
print(D1[22]['header'][0])
print(D1[100]['header'][0])
print(D1[25]['header'][0])


In [ ]:
# ##Approach 1
# from langchain.chat_models import ChatOpenAI
# from langchain.chains import create_extraction_chain
# schema = {
#     "properties": {
#         "disease outbreak": {"type": "string"},
#     },
#     "required": ["disease outbreak"],
# }

# # Input 
# inp = "Published Date: 2018-10-17 08:11:34 EDT Subject: PRO/AH/EDR> Undiagnosed die-off, hare - UK (02): (England) myxomatosis susp Archive Number: 20181017.6096127"
# # Run chain
# # Use model="gpt-3.5-turbo" for gpt 3.5. Its cheaper.
# llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model="gpt-4")
# chain = create_extraction_chain(schema, llm)
# chain.run(inp)
# #Didn't work well

In [ ]:
# ## Example 2
# inp = "Published Date: 2018-09-30 07:04:05 EDT Subject: PRO/EDR> Meningitis, meningococcal - USA (02): (CA) college, sg B Archive Number: 20180930.6061362"
# llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model="gpt-4")
# chain = create_extraction_chain(schema, llm)
# chain.run(inp)

In [ ]:
# Approach 2
inp = "Published Date: 2018-10-17 08:11:34 EDT Subject: PRO/AH/EDR> Undiagnosed die-off, hare - UK (02): (England) myxomatosis susp Archive Number: 20181017.6096127"
#chat([HumanMessage(content="Extract the disease outbreak from the following text. Only output the outbreak name. If the outbreak is known by other names give a comma separated list:" + D1[25]['header'][0])])
chatGPT35([HumanMessage(content="Extract the disease outbreak from the following text. " + "Only output the outbreak name." + inp )])
## This approach gives better results

In [ ]:
# Example 2
inp = "Published Date: 2018-09-30 07:04:05 EDT Subject: PRO/EDR> Meningitis, meningococcal - USA (02): (CA) college, sg B Archive Number: 20180930.6061362"
chatGPT35([HumanMessage(content="Extract the disease outbreak from the following text. " + "Only output the outbreak name." + inp )])

In [ ]:
# To see how good are LLMs for picking alternate names. Maybe not the best example.
chat([HumanMessage(content="Give alternative names of the following disease outbreak with confidence:" + "Sars-CoV-2")])

In [ ]:
xdata = np.array([(1, 2, 3), (4, 5, 6), (7, 8, 9)],
                dtype=[("a", "i4"), ("b", "i4"), ("c", "i4")])

In [8]:
import pandas as pd
import numpy as np
import datetime
df = pd.DataFrame({'floa': [np.nan],
                   'int': [np.nan],
                   'datetime': [pd.Timestamp('20180310')],
                   'string': ['foo']})
df = df.astype({"floa":"float64", "int":"Int64", "datetime":"datetime64[ns]", "string":"string[pyarrow]"})
dti = pd.to_datetime(
    ["1/1/2018", np.datetime64("2018-01-01"), datetime.datetime(2018, 1, 1)]
)

In [ ]:
import glob
import re
glob.glob('../*')
min([int(num) for num in re.findall(r'\d+','HI45_FG5')])

In [ ]:
max([1])

In [ ]:
[]+[0]

In [9]:
df

,floa,int,datetime,string
0,NaN,<NA>,2018-03-10,foo


In [ ]:
np.where(['floa' not in df.columns,'float' not in df.columns ])[0]

In [ ]:
from data.promed import ProMed

In [4]:
from data.promed import ProMed
PM = ProMed(PROMED_DIR)

In [ ]:
import glob
glob.glob('../*')

In [ ]:
PROMED_DIR = os.path.join('..','WHO_DON','Data','ProMED')

In [ ]:
PROMED_DIR


In [ ]:
import numpy as np
import pandas as pd
aa = np.array(["1bc", "defd", "eeeeeeeee"])
df = pd.DataFrame(aa)
#np.array([1,23,4], dtype='object')

In [ ]:
df.dtypes


In [ ]:
aa

In [5]:
PM.df

,ID,outbreakName,outbreakNameStd,countries,states,datePublished
0,0,None,None,None,None,None
1,1,None,None,None,None,None
2,2,None,None,None,None,None
3,3,None,None,None,None,None
4,4,None,None,None,None,None
...,...,...,...,...,...,...
64539,64539,None,None,None,None,None
64540,64540,None,None,None,None,None
64541,64541,None,None,None,None,None
64542,64542,None,None,None,None,None


In [4]:
[None]*9

[None, None, None, None, None, None, None, None, None]